In [2]:
from CephNet import CephNet

CephNet = CephNet()


CephNet(
  (ResNet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [ ]:
from torch import optim


# model.train()


def train(model, dataloader, loss_fn, optimizer, device, i):


    running_loss = 0.
    last_loss = 0.

    for batch_idx, (inputs, targets) in enumerate(dataloader):
        # Every data instance is an input + label pair
        inputs = inputs.to(device)
        target = targets.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        output = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(output, target)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f"Epoch: {i} | Loss: {epoch_loss}")

    return last_loss

In [3]:
from helper import  draw_batch

# for (images, landmarks,_) in train_dataloader:
#     print(f"Shape of Image Batch: {images.shape}")
#     print(f"Shape of Landmark Batch: {landmarks.shape} {landmarks.dtype}")
#     # draw_batch(images, landmarks)
#     output = CephNet.forward(images)
#     print(f"Shape of Output batch: {output.shape}")
#     break



Shape of Image Batch: torch.Size([5, 3, 512, 512])
Shape of Landmark Batch: torch.Size([5, 29, 2]) torch.float32
Shape of Output batch: torch.Size([5, 29, 2])


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from dataset import AarizDataset
from CephNet import CephNet,train,validate,test

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = CephNet()
model.to(device)


path = "Aariz Dataset"

batch_size = 5

training_data = AarizDataset( path ,"TRAIN")
val_data = AarizDataset( path ,"VALID")
test_data = AarizDataset( path ,"TEST")

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle=True)


num_epochs = 100
loss_fn = nn.SmoothL1Loss(beta=1.0)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)

for epoch in range (num_epochs):
    print(f"\n--- Epoch {epoch+1}/{num_epochs} ---")

    # 1. TRAIN
    # Returns (epoch_loss, avg_euclid)
    train_loss, train_euclid = train(model, train_dataloader, loss_fn, optimizer, device, epoch)

    # 2. VALIDATE
    # Returns (epoch_loss, avg_euclid)
    val_loss, val_euclid = validate(model, val_dataloader, loss_fn, device, epoch)

    # 3. UPDATE SCHEDULER
    # ReduceLROnPlateau watches the Validation Loss
    scheduler.step(val_loss)

    # 4. Check for overfitting
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_cephnet_model.pth")
        print(f"⭐ New Best Model Saved! (Val Loss: {val_loss:.6f})")

# ==========================================
# FINAL TEST PHASE (After Training Finishes)
# ==========================================
print("\n" + "="*30)
print("TRAINING COMPLETE. RUNNING FINAL TEST...")

# Load the weights from the BEST performing epoch
model.load_state_dict(torch.load("best_cephnet_model.pth"))

# Run the test function on the unseen Test Set
final_loss, final_mre = test(model, test_dataloader, loss_fn, device, "FINAL")










